In [ ]:
# instalação
!pip install yfinance==0.2.41
!pip install crewai==0.28.8
!pip install 'crewai[tools]'
!pip install langchain==0.1.20
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.38
!pip install duckduckgo-search==5.3.0

In [2]:
import json
import os
from datetime import datetime
from IPython.display import Markdown

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [3]:
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stocks prices for {ticket} from the last year about a specific sock from Yahoo Finance API",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [15]:
os.environ['OPENAI_API_KEY'] = "your_api_key"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [16]:
stock_price_analyst = Agent(
    role = "Senior stock price Analyst",
    goal = "Find the {ticket} stock price and analyses trends.",
    backstory = """You're highly experienced in analyzing the price of an specific
    stock and make predections about its future price.
    """,
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    allow_delegation = False,
    tools = [yahoo_finance_tool]
)

In [17]:
get_stock_price = Task(
    description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output = """Specify the current trend stock price - up, down or sideways.
    eg. stock = 'APPL, price UP'
    """,
    agent = stock_price_analyst
)

In [18]:
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [19]:
news_analyst = Agent(
    role = "Stock News Analyst",
    goal = """Create a short summary of the market news related to the stock {ticket} company.
    Specify the current trend - uo, down or sideways with the news context.
    For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear
    and 100 is extreme greed.
    """,
    backstory = """You're highly experienced in analyzing the market trends and news and have
    tracked assest for more the 10 years.
    
    you're also master level analyst in the tradicional markets and have deep understandig of
    human psychology.
    
    You understand new, theirs tittles and information, but you look at those with a
    health dose of skepticism. You consider also the source of the news articles.
    """,
    verbose = True,
    llm = llm,
    max_iter = 10,
    memory = True,
    allow_delegation = False,
    tools = [search_tool]
)

In [20]:
get_news = Task(
    description = f"""Take the stock and alwys include BTC to it (if not request).
    Use the search tool to search each one individually.
    
    The curremt date is {datetime.now()}.
    
    Compose the result into a helpfull report.
    """,
    expected_output = """A summary of the overall market and one sentence summary
    for each request asset. Include a fear/greed score for each asset based on the news.
    
    Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent = news_analyst
)

In [21]:
stock_analyst_writer = Agent(
    role = "Senior stock analysts writer",
    goal = """Analyze the trends price and news and write an insighfull compelling and
    informative 3 paragraph long newsletter based on the stock report and price trend.
    """,
    backstory = """You're widely accepted as the best stock analyst in the market.
    You understand complex concepts and create compelling stories and narratives that
    resonate with wider audiences.
    
    You understand macro factors and combine multiple theories.
    eg. cycle theory and fundamental analyses.
    You're able to hold multiple opinions when analyzing anything.
    """,
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    allow_delegation = True,
    tools = [search_tool]
)

In [22]:
write_analyses = Task(
    description = """Use the stock price trend and the stock news report to create
    an analyses and write the newsletter about the {ticket} company that is brief
    and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near
    future considerations?
    Include the previous analyses of stock trend and news summary.
    """,
    expected_output = """An eloquent 3 paragraph newsletter formated as markdown
    in an easy readable manner.

    It should contain:
    
    - 3 bullets executive summary.
    - Introduction - set the overall picture and spike up the interest.
    - Main part provides the meat of the analysis including the news summary and fear/greed scores.
    - Summary - key facts and concrete future trend prediction - up, down or sideways.
    """,
    agent = stock_analyst_writer,
    context = [get_stock_price, get_news]
)

In [23]:
crew = Crew(
  agents = [stock_price_analyst, news_analyst, stock_analyst_writer],
  tasks = [get_stock_price, get_news, write_analyses],
  verbose = True,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15  
)

2024-08-20 19:34:52,632 - 132686353936384 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [24]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information on the stock AAPL price history to determine the current trend. I should delegate the task of analyzing the stock price history to the Senior stock price Analyst and ask the Senior stock analysts writer to create a trend analysis based on the information gathered.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior stock price Analyst", "task": "Analyze the stock AAPL price history and determine the trend", "context": "Please analyze the historical stock price data of AAPL and determine the current trend (up, down, or sideways)"}

> Entering new CrewAgentExecutor chain...
I need to use the Yahoo Finance Tool to fetch the historical stock prices of AAPL and analyze the trend.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546356   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date      

In [25]:
list(results.keys())

['final_output', 'tasks_outputs']

In [26]:
results['final_output']

"# AAPL Newsletter\n\n## Executive Summary\n- AAPL stock has experienced recent fluctuations with a mix of positive and negative news.\n- Concerns about innovation lagging and slightly fearful sentiment persist in the market.\n- Investors should closely monitor the stock and consider a balanced trading approach.\n\n## Introduction\nThe recent news related to AAPL stock has been a rollercoaster of emotions for investors. From Warren Buffett's sell-off causing a 10% drop to stabilizing and beating expectations for its fiscal third quarter, the sentiment has been a whirlwind. As the only Magnificent 7 stock trading positively, AAPL is in a unique position amidst concerns about innovation lagging.\n\n## Main Analysis\nAnalyzing the stock price trend, it is evident that AAPL is currently trading sideways. Investors are cautious due to the recent fluctuations and slightly fearful sentiment. However, there are indications of further upside potential, presenting an opportunity for long-term in

In [27]:
len(results['tasks_outputs'])

3

In [28]:
Markdown(results['final_output'])

# AAPL Newsletter

## Executive Summary
- AAPL stock has experienced recent fluctuations with a mix of positive and negative news.
- Concerns about innovation lagging and slightly fearful sentiment persist in the market.
- Investors should closely monitor the stock and consider a balanced trading approach.

## Introduction
The recent news related to AAPL stock has been a rollercoaster of emotions for investors. From Warren Buffett's sell-off causing a 10% drop to stabilizing and beating expectations for its fiscal third quarter, the sentiment has been a whirlwind. As the only Magnificent 7 stock trading positively, AAPL is in a unique position amidst concerns about innovation lagging.

## Main Analysis
Analyzing the stock price trend, it is evident that AAPL is currently trading sideways. Investors are cautious due to the recent fluctuations and slightly fearful sentiment. However, there are indications of further upside potential, presenting an opportunity for long-term investors. The fear/greed score of 40 reflects the mixed sentiment in the market, signaling a need for a balanced and strategic approach.

## Summary
In conclusion, AAPL stock is at a crossroads with conflicting factors influencing its performance. While there are concerns about innovation lagging and a slightly fearful sentiment, there are also positive aspects to consider. Investors are advised to stay vigilant, monitor market developments closely, and approach trading with caution. The long-term outlook for AAPL remains positive, but short-term volatility may impact its performance.